In [102]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import utils.featurizer as ft
import utils.encodeador as my_encoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train = pd.read_csv("data/train.csv")

In [3]:
test = pd.read_csv("data/test.csv")

In [4]:
df_test = pd.read_csv("data/test.csv")

In [5]:
drop_cols = ['titulo','fecha', 'ciudad', 'idzona', 'descripcion', 'direccion', 'lat', 'lng', 'id']

In [6]:
train = ft.featurizer(train)
train = le.label_encoder(train)

In [7]:
test = ft.featurizer(test)
test = le.label_encoder(test)

In [8]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 21 columns):
tipodepropiedad                  240000 non-null int64
provincia                        240000 non-null int64
antiguedad                       240000 non-null float64
habitaciones                     240000 non-null float64
garages                          240000 non-null float64
banos                            240000 non-null float64
metroscubiertos                  240000 non-null float64
metrostotales                    240000 non-null float64
gimnasio                         240000 non-null float64
usosmultiples                    240000 non-null float64
piscina                          240000 non-null float64
escuelascercanas                 240000 non-null float64
centroscomercialescercanos       240000 non-null float64
year                             240000 non-null float64
mes                              240000 non-null float64
dia_del_mes                      24000

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [10,100, 200, 300, 1000]
}

In [12]:
randomForest = RandomForestRegressor(**param_grid)

In [13]:
%%time
grid_search = GridSearchCV(estimator = randomForest, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring='neg_mean_absolute_error')

CPU times: user 108 µs, sys: 21 µs, total: 129 µs
Wall time: 35.8 µs


In [14]:
%%time
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.5min
/home/braianvillalba/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 26.4min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 69.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 128.5min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 205.0min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 225.5min finished


CPU times: user 3min 1s, sys: 5.88 s, total: 3min 7s
Wall time: 3h 48min 21s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=[True], criterion='mse',
                                             max_depth=[80, 90, 100, 110],
                                             max_features=[2, 3],
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=[3, 4, 5],
                                             min_samples_split=[8, 10, 12],
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=[10, 100, 200, 300,
                                                           1000],
                                             n_j...lse,
                                             random_state=None, verbose=0,
                    

In [15]:
best_grid = grid_search.best_estimator_

In [18]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [19]:
pred = best_grid.predict(X_test)

In [20]:
mean_absolute_error(y_test,pred)

690631.0164816774

In [22]:
pred = best_grid.predict(test)

In [27]:
res = pd.DataFrame(pred, index=df_test.id, columns=['target'])
display(res.head())
res.to_csv("data/workshop-randomforest-v3_0.csv", header=True)

,target
id,
4941,5.154050e+06
51775,1.167726e+06
115253,2.534278e+06
299321,1.584989e+06
173570,6.446902e+05


In [25]:
feature_importances = pd.DataFrame(best_grid.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [26]:
feature_importances

,importance
metroscubiertos,0.221503
metrostotales,0.131224
banos,0.104513
metro_promedio_por_cuadricula,0.099263
provincia,0.070121
is_DF,0.068235
tipodepropiedad,0.056662
aire_libre,0.040394
garages,0.034971
habitaciones,0.033659


In [28]:
##CON ONE HOT ENCODING


In [103]:
train = pd.read_csv("data/train.csv")

In [104]:
test = pd.read_csv("data/test.csv")

In [105]:
df_test = pd.read_csv("data/test.csv")

In [106]:
drop_cols = ['titulo','fecha', 'ciudad', 'idzona', 'descripcion', 'direccion', 'lat', 'lng', 'id']

In [107]:
train = ft.featurizer(train)
test = ft.featurizer(test)

In [108]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [109]:
X_oh,test_oh = le.oneHotEncoder(X,test)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X_oh, y, test_size=0.25, random_state=42)

In [113]:
some_param = {'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400
}

In [114]:
randomForest = RandomForestRegressor(**some_param)

In [115]:
%%time
randomForest.fit(X_train,y_train)

CPU times: user 10min 35s, sys: 211 ms, total: 10min 36s
Wall time: 10min 36s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=70,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [119]:
predOh = randomForest.predict(X_test)

In [120]:
mean_absolute_error(y_test,predOh)

666225.5499975738

In [121]:
pred = randomForest.predict(test_oh)

In [123]:
res = pd.DataFrame(pred, index=df_test.id, columns=['target'])
display(res.head())
res.to_csv("data/workshop-randomforest-v3_1.csv", header=True)

,target
id,
4941,5.376694e+06
51775,9.959708e+05
115253,2.572719e+06
299321,1.506746e+06
173570,6.616562e+05
